# Detección de Autos con Object Detection Api [TensorFlow]

In [56]:
import numpy as np
import cv2 as cv
import tensorflow as tf
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw
import ipywidgets as widgets
import matplotlib.pyplot as plt

from pathlib import Path
from IPython.display import HTML
from collections import defaultdict
from io import StringIO

In [57]:
# Comandos para crear las librerias de object detection
# Ejecutarlas mediante shell
#$ cd models/research
#$ ./bin/protoc object_detection/protos/*.proto --python_out=.
#$ export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

<b>Ubicar dirección del video</b>

In [58]:
PATH_VIDEO = Path("../Videos/Prueba1.mp4")
if not PATH_VIDEO.exists():
    print("El video {} no existe".format(PATH_VIDEO))

<b>Establecer los datos del modelo</b>

In [59]:
#direccion del modelo
PATH_MODEL = Path('../ssd_mobilenet_v1_coco_2017_11_17/saved_model/')
if not PATH_MODEL.is_dir():
    print("La carpeta del modelo {} no existe".format(PATH_MODEL))

<b>Función para cargar el modelo</b>

In [60]:
def cargar_modelo(path_modelo):
    model = tf.saved_model.load(str(path_modelo))
    model = model.signatures['serving_default']  
    return model

<b>Función para Inferir un Frame/Imagen</b>

In [61]:
def inferir_imagen(modelo, img_Inicial):
    
    #Convertir la imagen en arreglo
    img_array = np.asarray(img_Inicial)
    
    # La entrada debe ser un tensor, convertirlo usando 'tf.convert_to_tensor'
    input_tensor = tf.convert_to_tensor(img_array)
    
    # El modelo espera las imagenes, asi que se agrega un eje con 'tf.newaxis'
    input_tensor = input_tensor[tf.newaxis,...]
    
    # Ejecutar la inferencia
    output_dict = modelo(input_tensor)
    
    num_detections = int(output_dict.pop('num_detections'))
    #Reducimos el numero de detecciones a num_detections = 7
    num_detections = 7
    
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    
    # detection_classes deben ser ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
    toReplace = np.where((output_dict['detection_classes'] != 3 ))
    #Reemplazamos los scores/puntajes de cualquier otro objeto para ignorarlo
    output_dict['detection_scores'].flat[toReplace] = 0.00111111
    
    return output_dict

<pre>
    <b>Función para convertir las coordenadas del modelo</b><br>
    Las funcion convertira las coordenadas obtenidas en
    output_dict['detection_boxes'] de la inferencia del frame, 
    para retornarlas en formato de pixeles  a escala de la imagen
</pre>

In [62]:
def calcular_Coordenadas(arr_coor,img_ancho,img_alto):
    #[(248,84),(248,139),(396,139),(396,84),(248,84)]
    (left, right, top, bottom) = (arr_coor[1] * img_ancho, arr_coor[3] * img_ancho,
                                  arr_coor[0] * img_alto, arr_coor[2] * img_alto)

    return [(left, top), (left, bottom), (right, bottom), (right, top), (left, top)]

In [63]:
def getFrame(modelo, frame_base):
    img_inicial = np.array(frame_base)
    result_inferencia = inferir_imagen(modelo, img_inicial)
    alto, ancho = frame_base.shape[0], frame_base.shape[1]
    
    #Si es un auto
    if (result_inferencia['detection_classes'][0] == 3) & (result_inferencia['detection_scores'][0] > float(0.45000000)):
        #print(result_inferencia['detection_scores'][0])
        
        coordenadas = calcular_Coordenadas(result_inferencia['detection_boxes'][0],ancho,alto)
    
        img_Pil = Image.fromarray(img_inicial)
        marco = ImageDraw.Draw(img_Pil)
        marco.line(coordenadas, fill="green",width=3) 
        #Regresa el freme con la deteccion del auto
        return np.asarray(img_Pil)
    
    #Regresa el frame igual
    return frame_base

In [76]:
# Cargar el modelo de detección
MODEL_DETECTION = cargar_modelo(PATH_MODEL)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [81]:
cap = cv.VideoCapture(str(PATH_VIDEO))

In [82]:
#Cargar codec seleccionado (*mp4 es por defecto util en opencv para procesar videos en formatos MP4 )
fourcc = cv.VideoWriter_fourcc(*'mp4v')

In [83]:
#Establecer la configuracion del escritor del Video de OpenCV (VideoWriter)
# formato (VIDEO DESTINO, CODEC, FPS, Dimensiones de los frames)
out = cv.VideoWriter('../Videos/Video_p4.mp4',fourcc, 29.04, (640,352))

In [84]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        #grayFrame0 = cv.cvtColor(frame, cv.COLOR_BGR2GRAY,0)
        #grayFrame = cv.cvtColor(grayFrame0,cv.COLOR_GRAY2BGR,0)
        
        #Convertimos el frame a escala de colores
        grayFrame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        grayFrame = cv.cvtColor(grayFrame,cv.COLOR_GRAY2RGB)
        #Obtenemos el Frame final con la deteccion
        finalFrame = getFrame(MODEL_DETECTION, grayFrame)
        #Antes de escribirl se necesita en BGR
        finalFrame = cv.cvtColor(finalFrame, cv.COLOR_RGB2BGR)
        
        #Escribir el frame en el video
        out.write(finalFrame)
    else:
        break
        
# Liberar la memoria utilizada
# del procesamiento del video(cap)
# y su escritura (out)
cap.release()
out.release()
print("Video Guardado")

Fin del procesamiento


In [33]:
#comando para verificar los codecs de video [FFMPEG]
#!ffmpeg -encoders

In [87]:
#Comando para converion de video [El proceso puede tardar unos segundos]
!ffmpeg -i ../Videos/Video_p4.mp4 -c:v libx264 ../Videos/Video_pf.mp4 -y 2> /dev/null
print("Conversión Completa")

Conversión Completa


In [88]:
#Cargar el video codificado en la Notebook
HTML("""
<style> video {  max-width: 100%;  height: auto; } </style>
    <video alt="test" controls autoplay>
        <source src="../Videos/Video_pf.mp4" type="video/webm">
    </video>
""")